# Cifrado de bloque: modos de cifrado AES

Las prácticas de este curso las haremos en Python, conde tenemos dos posibilidades para probar criptografía: el paquete `PyCryptodome` y el paquete `cryptography`. Ambos son opciones válidas e intercambiables. Las prácticas de este curso las haremos con PyCryptodome. Puedes encontrar la ayuda en: https://pycryptodome.readthedocs.io/en/latest/

Si estás siguiendo estas notas en tu PC, puedes instalarlo con: `python3 -m pip install pycryptodome`. Si las estás siguiendo en Colab, ejecuta la siguiente línea:

In [1]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.4 MB/s eta 0:00:00




AES se puede utilizar en varios modos. Vamos a verlos en esta actividad.

Vamos a crear:
- un mensaje de 128 bits, el tamaño de bloque de AES.
- una clave k de 128 bits que usaremos durante todo el ejercicio.

In [2]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from base64 import b64encode, b64decode

m = b'abcdefghabcdefgh'
k = get_random_bytes(16)
print(f'Mensaje: "{m}" Tamaño={len(m) * 8} bits')
print(f'Clave: {b64encode(k)} Tamaño={len(k) * 8} bits')

Mensaje: "b'abcdefghabcdefgh'" Tamaño=128 bits
Clave: b'WqrwLp/uDUNCNr3JbNG6Sg==' Tamaño=128 bits


## Modo ECB

El primer modo que veremos es "ECB": cada bloque se cifra de forma independiente, sin realimentación con los demás.

![center w:30em](https://github.com/Juanvvc/crypto/blob/master/ejercicios/03/images/ECB_encryption.svg?raw=1)

Vamos a ciframos tres veces el mismo mensaje. Observa que no hay memoria, y que cifrar dos veces el mismo mensaje con la misma clave produce el mismo texto cifrado. Por eso no se debe usar nunca el modo ECB: un atacante que observa las comunicaciones quizá no sepa qué estamos diciendo, pero sí que sabe que estamos repitiendo el mensaje y eso puede ser suficiente para sus objetivos.

**Nunca debe usarse AES en modo ECB**


In [5]:
cipher = AES.new(k, AES.MODE_ECB)
c1 = cipher.encrypt(m)
c2 = cipher.encrypt(m)
c3 = cipher.encrypt(m)
print(b64encode(c1))
print(b64encode(c2))
print(b64encode(c3))

b'7XcCU0tCV3pYhwyL+wiVcg=='
b'7XcCU0tCV3pYhwyL+wiVcg=='
b'7XcCU0tCV3pYhwyL+wiVcg=='


In [6]:
decipher = AES.new(k, AES.MODE_ECB)
m1 = decipher.decrypt(c1)
m2 = decipher.decrypt(c2)
m3 = decipher.decrypt(c3)
print(m1)
print(m2)
print(m3)

b'abcdefghabcdefgh'
b'abcdefghabcdefgh'
b'abcdefghabcdefgh'


## Modo CBC

En el modo CBC, hay realimentación entre bloques y existe un vector de inicialización

![](https://github.com/Juanvvc/crypto/blob/master/ejercicios/03/images/CBC_encryption.svg?raw=1)

Ciframos dos veces el mismo mensaje. Observa que tenemos que crear un IV (Vector de Inicialización), y que este IV se lo tenemos que enviar al receptor. El envío del IV se hace durante el primer mensaje, antes de que el canal sea seguro, pero no hay problemas en que un atacante conozca el IV.

In [7]:
iv = get_random_bytes(16)
cipher = AES.new(k, AES.MODE_CBC, iv=iv)
c1 = cipher.encrypt(m)
c2 = cipher.encrypt(m)
print(b64encode(c1))
print(b64encode(c2))

b'HcpsV9yRVSncbRWxtZpSMQ=='
b'UdwT/047CKsAYbOgATijLw=='


***

<font color="green">
PREGUNTA: ahora los dos cifrados son diferentes a pesar de que estamos cifrando el mismo mensaje. ¿Por qué sucede eso?
</font>

***

<font color="blue">
RESPUESTA: añade aquí tu respuesta
</font>

***

Descifrado: necesita la clave y el IV. La clave es secreta y el receptor tiene que haberla recibido por un canal secreto (lo veremos) pero el IV puede recibirse sin protección al inicio de la comunicación.

In [8]:
decipher = AES.new(k, AES.MODE_CBC, iv=iv)
m1 = decipher.decrypt(c1)
m2 = decipher.decrypt(c2)
print(m1)
print(m2)

b'abcdefghabcdefgh'
b'abcdefghabcdefgh'


## Ejercicio **opcional**

- ¿Puedes programar el modo CBC a partir del modo ECB? ECB es la caja AES básica, así que es posible programar (¡como ejercicio solamente!) el modo CBC como composición de ECB
- ¿Puedes programar los demás modos?

Ejemplo de solución (solo parte de cifrado) de la primera pregunta. Observa que el resultado es el mmismo de antes al cifrar m en modo CBC

In [9]:
from Crypto.Util.strxor import strxor

class AES_CBC():
    def __init__(self, k, iv):
        self.iv = iv
        self.cipher = AES.new(k, AES.MODE_ECB)
    def encrypt(self, msg):
        # primero hacemos XOR del mensaje con el IV que tenemos
        m = strxor(msg, self.iv)
        c = self.cipher.encrypt(m)
        # para la siguiente ronda, el IV es el propio texto cifrado
        self.iv = c
        return c

k = get_random_bytes(16)
iv = get_random_bytes(16)
mycbc = AES_CBC(k, iv)
print(b64encode(mycbc.encrypt(m)))
print(b64encode(mycbc.encrypt(m)))

b'rJP+Swkvh+LgOSxKMJ9uvQ=='
b'5aSpH4sFY9UO3ylAntAznQ=='


## Padding

¿Qué pasa si tenemos que enviar mensajes más cortos que la longitud de bloque de AES? Entonces tenemos que usar algún algoritmo de padding. Es decir: marcar la longitud del mensaje original. Observa que solo podemos enviar bloques de 128 bits, y si intentamos enviar bloques más cortos o más largos saltará un error:

In [10]:
c1 = cipher.encrypt(b'mensaje corto')

ValueError: Data must be padded to 16 byte boundary in CBC mode

Con Cryptodome podemos usar las funciones `pad()` y `unpad()`

Observa: en este ejemplo no especificamos un IV al crear el cipher, pero ya sabes que en modo CBC siempre existe un IV. Si no lo especificamos en creación, los escogerá al azar.

In [11]:
from Crypto.Util.Padding import pad, unpad

# mensaje corto
m = b'1234'
cipher = AES.new(k, AES.MODE_CBC)
c = cipher.encrypt(pad(m, AES.block_size))
print({'iv':b64encode(cipher.iv), 'ciphertext':b64encode(c)})

{'iv': b'vYhOcjSOsZZJlk+PZ7hEdQ==', 'ciphertext': b'NP5TjQJuFGhPefgbVIyfTQ=='}


Recepción:

In [12]:
decipher = AES.new(k, AES.MODE_CBC, cipher.iv)
pt = unpad(decipher.decrypt(c), AES.block_size)
print("The message was: ", pt)

The message was:  b'1234'


¿Qué pasa si no usamos unpad? AES es un cifrado de bloque, así que los mensajes en AES tienen obligatoriamente un tamaño igual al bloque AES (128 bits), así que vemos el *padding* que sobra. Las función *unpad()* nos hubiese cortado esos bytes sobrantes.

(Observa: tenemos que volver a recrear el decipher, porque tiene memoria y queremos volver a descifrar el mismo mensaje)

In [13]:
decipher = AES.new(k, AES.MODE_CBC, cipher.iv)
pt = decipher.decrypt(c)
print(f"The message was: {pt} (longitud {len(pt) * 8} bits)")

The message was: b'1234\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c' (longitud 128 bits)


***

<font color="red">
PREGUNTA: vimos que ChaCha20 podía resincronizarse si se perdían mensajes. ¿AES en modo CBC puede resincronizarse si se pierden mensajes? ¿Qué modo de AES permitiría resincronizarse?
</font>

***

<font color="blue">
RESPUESTA: añade aquí tu respuesta
</font>

***